# Loosely based on Chapter 8 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by F. Chollet

In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
path = keras.utils.get_file('nietzsche.txt', 
                           origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with open(path) as f:
    text = f.read().lower()

606208/600901 [==============================] - 3s 5us/step


In [3]:
print('Corpus length:', len(text))

Corpus length: 600893


In [5]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))

print('Unique chars:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

Number of sequences: 200278
Unique chars: 57


### Vectorize

In [8]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    

In [9]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))

In [11]:
def sample(preds, temperate=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.max(probas)

In [ ]:
import random, sys

for epoch in range(1, 2):
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--Seed--\n', generated_text)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('--Temperature--\n', temperature)
        sys.stdout.write(generated_text)
        for i in range(100):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text +=next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

Epoch 1/1
125312/200278 [=================>............] - ETA: 1:21 - loss: 1.5141